In [1]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt 
import netCDF4 as nc
from netCDF4 import Dataset
import geopandas as gpd
import pandas as pd
import seaborn as sns
import random
from sklearn.model_selection import train_test_split
import xarray as xr

In [2]:
# Import all standardized features 

sinmod = xr.open_dataset('/cluster/home/maikents/features_midnor_2019/ready_for_training/SINMOD_bottom_features_emod_grid_standardized.nc')
current_aspect_angle = xr.open_dataset('/cluster/home/maikents/features_midnor_2019/ready_for_training/standardized_single_features/standardized_current_aspect_angle.nc')
emod = xr.open_dataset('/cluster/home/maikents/features_midnor_2019/ready_for_training/EMOD_standardized.nc')


In [3]:
# EMOD first
emod_df = emod.to_dataframe().reset_index()
print(emod_df.columns)
emod_df = emod_df.dropna()


Index(['x', 'y', 'aspect_cos', 'aspect_sin', 'bathymetry', 'broad_BPI',
       'fine_BPI', 'log_ruggedness', 'slope'],
      dtype='object')


In [5]:
# Get the "stats sinmod features"

mean = sinmod.sel(stat='mean')
minimum = sinmod.sel(stat='10th_percentile')
maximum = sinmod.sel(stat='90th_percentile')

mean_data_df = mean.to_dataframe().reset_index()
min_data_df = minimum.to_dataframe().reset_index()
max_data_df = maximum.to_dataframe().reset_index()

mean_data_df = mean_data_df.rename(columns={col: f"{col}_mean" for col in mean_data_df.columns if col not in ['x', 'y']})
min_data_df = min_data_df.rename(columns={col: f"{col}_10th_percentile" for col in min_data_df.columns if col not in ['x', 'y']})
max_data_df = max_data_df.rename(columns={col: f"{col}_90th_percentile" for col in max_data_df.columns if col not in ['x', 'y']})



In [6]:
print(list(sinmod.variables))

['stat', 'x', 'y', 'spatial_ref', 'bottom_temperature_sundahl_features', 'bottom_salinity_features', 'bottom_current_features', 'bottom_statistical_northness_features', 'bottom_statistical_eastness_features']


In [6]:
sinmod_df = sinmod[['x', 'y', 'bottom_temperature_sundahl_features', 'bottom_salinity_features',
                                  'bottom_current_features', 'bottom_statistical_northness_features',
                                  'bottom_statistical_eastness_features']].to_dataframe().reset_index()


In [7]:
sinmod_df['current_aspect_angle'] = current_aspect_angle

In [8]:
sinmod_df = sinmod_df.dropna()

In [12]:
print(mean_data_df.columns)
print(min_data_df.columns)
print(max_data_df.columns)

Index(['x', 'y', 'stat_mean', 'spatial_ref_mean',
       'bottom_temperature_sundahl_features_mean',
       'bottom_salinity_features_mean', 'bottom_current_features_mean',
       'bottom_statistical_northness_features_mean',
       'bottom_statistical_eastness_features_mean'],
      dtype='object')
Index(['x', 'y', 'stat_10th_percentile', 'spatial_ref_10th_percentile',
       'bottom_temperature_sundahl_features_10th_percentile',
       'bottom_salinity_features_10th_percentile',
       'bottom_current_features_10th_percentile',
       'bottom_statistical_northness_features_10th_percentile',
       'bottom_statistical_eastness_features_10th_percentile'],
      dtype='object')
Index(['x', 'y', 'stat_90th_percentile', 'spatial_ref_90th_percentile',
       'bottom_temperature_sundahl_features_90th_percentile',
       'bottom_salinity_features_90th_percentile',
       'bottom_current_features_90th_percentile',
       'bottom_statistical_northness_features_90th_percentile',
       'bottom_

In [13]:
mean_data_df = mean_data_df.drop(['stat_mean', 'spatial_ref_mean', 'bottom_statistical_northness_features_mean','bottom_statistical_eastness_features_mean'], axis=1)
min_data_df = min_data_df.drop(['stat_10th_percentile', 'spatial_ref_10th_percentile', 'bottom_statistical_northness_features_10th_percentile', 'bottom_statistical_eastness_features_10th_percentile', 'bottom_current_features_10th_percentile'], axis=1)
max_data_df = max_data_df.drop(['stat_90th_percentile', 'spatial_ref_90th_percentile','bottom_statistical_northness_features_90th_percentile', 'bottom_statistical_eastness_features_90th_percentile'], axis=1)

In [20]:
print(sinmod_df.columns)

Index(['x', 'y', 'stat', 'bottom_temperature_sundahl_features',
       'bottom_salinity_features', 'bottom_current_features',
       'bottom_statistical_northness_features',
       'bottom_statistical_eastness_features', 'current_aspect_angle'],
      dtype='object')


In [21]:
sinmod_df = sinmod_df.merge(mean_data_df, on=['x', 'y'], suffixes=('', '_mean'))

In [22]:
sinmod_df = sinmod_df.merge(min_data_df, on=['x', 'y'], suffixes=('_mean', '_10th_percentile'))

In [23]:
sinmod_df = sinmod_df.merge(max_data_df, on=['x', 'y'], suffixes=('_10th_percentile', '_90th_percentile'))

In [25]:
sinmod_df = sinmod_df.dropna(axis=0, how='any')

In [29]:
total_df = sinmod_df.merge(emod_df, on=['x', 'y'])

In [30]:
print(total_df.columns)

Index(['x', 'y', 'stat', 'bottom_temperature_sundahl_features',
       'bottom_salinity_features', 'bottom_current_features',
       'bottom_statistical_northness_features',
       'bottom_statistical_eastness_features', 'current_aspect_angle',
       'bottom_temperature_sundahl_features_mean',
       'bottom_salinity_features_mean', 'bottom_current_features_mean',
       'bottom_temperature_sundahl_features_10th_percentile',
       'bottom_salinity_features_10th_percentile',
       'bottom_temperature_sundahl_features_90th_percentile',
       'bottom_salinity_features_90th_percentile',
       'bottom_current_features_90th_percentile', 'aspect_cos', 'aspect_sin',
       'bathymetry', 'broad_BPI', 'fine_BPI', 'log_ruggedness', 'slope'],
      dtype='object')


In [ ]:
total_df.to_parquet('/cluster/home/maikents/coral-mapping/processed_data/env_df_no_nan.parquet')